In [1]:
import sys
import os
file_path = os.path.dirname(os.path.realpath(__file__))
sys.path.append(os.path.join(file_path,'../build'))
import numpy as np
import libry as ry
import time
import cv2 as cv

**ry-c++-log** ry.cpp:init_LogToPythonConsole:34(0) initializing ry log callback

**ry-c++-log** util.cpp:initCmdLine:545(1) ** cmd line arguments: 'rai-pybind -python '
** INFO:ry.cpp:init_LogToPythonConsole:34(0) initializing ry log callback

**ry-c++-log** util.cpp:initCmdLine:549(1) ** run path: '/home/vasko/Documents/TUB3/AI_Robotics/robotics-course-prev/course3-Simulation'** INFO:util.cpp:initCmdLine:545(1) ** cmd line arguments: 'rai-pybind -python '


**ry-c++-log** graph.cpp:initParameters:1379(1) ** parsed parameters:
{python}

** INFO:util.cpp:initCmdLine:549(1) ** run path: '/home/vasko/Documents/TUB3/AI_Robotics/robotics-course-prev/course3-Simulation'

** INFO:graph.cpp:initParameters:1379(1) ** parsed parameters:
{python}



In [2]:
# OPTIONS
use_impulse = False

def add_box(C):
    f = C.addFrame("object")
    f.setShape(ry.ST.ssBox, [.06, .06, .15, 0.01])
    f.setPosition([0., .1, .8])
    f.setColor([1,0,0])
    f.setQuaternion( [  0, 0, 0.7071068 , 0.7071068 ] )
    f.setContact(1)
    f.setMass(1)
    f2 = C.addFrame("refobject", parent="object")
    f2.setShape(ry.ST.marker, [.1])


RealWorld = ry.Config()
C = ry.Config()
RealWorld.addFile("../../robotics-course/scenarios/pandasTable.g")
C.addFile("../../robotics-course/scenarios/pandasTable.g")
add_box(RealWorld)
add_box(C)

# you can also change the shape & size


D = C.view()



S = RealWorld.simulation(ry.SimulatorEngine.bullet, True)
S.addSensor("camera")
q0 = S.get_q()

if use_impulse:
    S.addImp(ry.ImpType.objectImpulses, ['object'], [])

tau = .01


# wait for the object to fall down 
for t in range(50):
    time.sleep(tau)
    S.step([], tau, ry.ControlMode.none)


gripping = False
grasped = False

C.setJointState(S.get_q())

# cheat perception 
objectPosition = RealWorld.frame("object").getPosition()
objectRotationMatrix = RealWorld.frame("object").getRotationMatrix()
objectQuaternion = RealWorld.frame("object").getQuaternion()

C.getFrame("object").setPosition(objectPosition)
C.getFrame("object").setQuaternion(objectQuaternion)

longAxis = objectRotationMatrix[:,2]

In [3]:
komo = C.komo_path(1, 40, 2., False)

komo.addObjective(
    [1], ry.FS.positionRel, [
        "R_gripperCenter", "object"], ry.OT.eq, 1e2, [0.,0.0,.0])


komo.addObjective([.5, 1.], ry.FS.positionRel, [
                  "R_gripperCenter", "object"], ry.OT.sos, 1e2 * np.array([1, 0, 0]))


# i can define a pregrasp position
komo.addObjective([.6, .8], ry.FS.positionRel, [
                  "R_gripperCenter", "object"], ry.OT.sos, 1e2  , [0, 0 , .1])

# NOTE: I can also use:
# komo.addObjective([1.], ry.FS.vectorY, [
#                   "R_gripperCenter"], ry.OT.sos, 1e1, longAxis )

komo.addObjective([1.], ry.FS.scalarProductYZ, [
                  "object","R_gripperCenter"], ry.OT.sos, 1e2, [1] )


komo.addObjective([1.], ry.FS.scalarProductYZ, [
                  "R_gripperCenter", "object"], ry.OT.sos, 1e3, [1] )

komo.addObjective([1.], ry.FS.scalarProductXY, [
                 "object",  "R_gripperCenter" ], ry.OT.eq, 1e2  )






# collision avoidance
komo.addObjective(
    [],
    ry.FS.accumulatedCollisions, [],
    ry.OT.eq, 1e1 )

komo.optimize()
komo.getReport()

** KOMO::run solver:sparse collisions:0 x-dim:560 T:40 k:2 phases:1 stepsPerPhase:40 tau:0.05  #timeSlices:42 #totalDOFs:560 #frames:4662
** optimization time:0.071761 (kin:0.002589 coll:0 feat:0.037734 newton: 0.017632) setJointStateCount:38
   sos:4.71803 ineq:0 eq:0.000354919


{'F_qItself/2-#28': {'order': 2.0, 'type': 'sos', 'sos': 4.0398664101368995},
 'F_PositionRel/0-R_gripperCenter-object': {'order': 0.0,
  'type': 'sos',
  'sos': 0.5230645248871332},
 'F_ScalarProduct/0-object-R_gripperCenter': {'order': 0.0,
  'type': 'eq',
  'eq': 9.166707040271505e-06},
 'F_ScalarProduct/0-R_gripperCenter-object': {'order': 0.0,
  'type': 'sos',
  'sos': 9.329866016633627e-07},
 'F_AccumulatedCollisions/0-#111': {'order': 0.0, 'type': 'eq'},
 'sos': 4.718031348290135,
 'ineq': 0.0,
 'eq': 0.00035491897905543066,
 'f': 0.0}

In [ ]:
gripping = False
grasped = False

S.setMoveto(komo.getPath_qOrg(), 2)

for t in range(3000):

    time.sleep(tau)
    C.setJointState(S.get_q())

    # cheat perception best
    
    C.setFrameState(S.getState()[0])
    # objectPosition = RealWorld.frame("object").getPosition()
    # objectRotationMatrix = RealWorld.frame("object").getRotationMatrix()
    # objectQuaternion = RealWorld.frame("object").getQuaternion()

    # C.getFrame("object").setPosition(objectPosition)
    # C.getFrame("object").setQuaternion(objectQuaternion)

    longAxis = objectRotationMatrix[:,2]

#     if gripping:
#         S.setMoveto(S.get_q(),tau, append=False)

    y, J = C.evalFeature(ry.FS.positionRel, ["R_gripperCenter", "object"])
    
    # print( "error grasp" , np.linalg.norm(y))
    if not gripping and np.linalg.norm(y) < .015:
        S.closeGripper("R_gripper", speed=1.)
        print("setting gripping to true")
        gripping = True
        break
        S.setMoveto(S.get_q(),tau, append=False)

    if gripping and S.getGripperWidth("R_gripper") < 1e-2:
        print("FAILED!")
        S.openGripper("R_gripper", speed=1.)
        gripping = False
        break

    if gripping and S.getGripperIsGrasping("R_gripper"):
        print("GRASPED!")
        grasped = True
        break

    S.step([], tau, ry.ControlMode.spline)

**ry-c++-log** pairCollision.cpp:libccd:287(0) WARNING: called MPR penetration for non intersecting meshes...

